## **Pomocne moduly**

In [41]:
import itertools
import math
import numpy as np
import UQpy
from scipy.special import comb
from UQpy.distributions import Uniform, JointIndependent
from UQpy.surrogates.polynomial_chaos import PolynomialChaosExpansion, TotalDegreeBasis, LeastSquareRegression
from UQpy.sensitivity import PceSensitivity
from sklearn.metrics import mean_squared_error, r2_score
import time  

Generování multiindexových množin

In [44]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb  6 12:57:36 2023

@author: D. Loukrezis
"""

import itertools
import math
import numpy as np
from scipy.special import comb


def setsize(N, w):
    """Vypočítá počet PCE polynomů pro N dimenzí a maximální stupeň w pomocí kombinatoriky"""
    return int(comb(N+w-1, N-1))    


def td_set_recursive(N, w, rows):
    """Rekurzivní funkce pro generování multiindexů totálního stupně"""
    if N == 1:  # Základní případ - 1D
        subset = w*np.ones([rows, 1])
    else:
        if w == 0:  # Nulový stupeň - všechny nuly
            subset = np.zeros([rows, N])
        elif w == 1:  # Stupeň 1 - jednotková matice
            subset = np.eye(N)
        else:
            # Inicializace podmatice
            subset = np.empty([rows, N])
            
            # Počáteční řádek podmatice
            row_start = 0
            
            # Iterace přes polynomiální stupně
            for k in range(0, w+1):
                
                # Počet řádků podmatice
                sub_rows = setsize(N-1, w-k)
                
                # Aktualizace koncového řádku
                row_end = row_start + sub_rows - 1
                
                # První sloupec - nastaví hodnotu k
                subset[row_start:row_end+1, 0] = k*np.ones(sub_rows)
                
                # Rekurzivní volání pro zbylé dimenze
                subset[row_start:row_end+1, 1:] = td_set_recursive(N-1, w-k, 
                                                              sub_rows)
                                                                     
                # Aktualizace indexu řádku
                row_start = row_end + 1
    
    return subset


def td_multiindex_set(N, w):
    """Generuje multiindex set totálního stupně pro N parametrů a max stupeň w"""
    
    # Velikost množiny multiindexů
    td_size = int(comb(N+w, N))
    
    # Inicializace výsledné množiny
    midx_set = np.empty([td_size, N])
    
    # Počáteční řádek
    row_start = 0
    
    # Iterace přes polynomiální stupně
    for i in range(0, w+1):
        
        # Výpočet počtu řádků pro daný stupeň
        rows = setsize(N, i)
        
        # Aktualizace koncového řádku
        row_end = rows + row_start - 1
        
        # Rekurzivní generování části množiny
        midx_set[row_start:row_end+1, :] = td_set_recursive(N, i, rows)
        
        # Aktualizace počátečního řádku
        row_start = row_end + 1
        
    return midx_set.astype(int)  # Převod na celá čísla


def tp_multiindex_set(N, w):
    """Generuje tenzorový součin multiindex set pro N parametrů a max stupeň w"""
    orders = np.arange(0, w+1, 1).tolist()  # Seznam stupňů 0 až w
    if N == 1:  # Speciální případ pro 1D
        midx_set = np.array(list(map(lambda el:[el], orders)))
    else:
        # Generování všech kombinací (kartézský součin)
        midx = list(itertools.product(orders, repeat=N))
        midx = [list(elem) for elem in midx]
        # Výpočet součtů pro každý multiindex
        midx_sums = [int(math.fsum(midx[i])) for i in range(len(midx))]
        # Seřazení podle součtu (totální stupeň)
        midx_sorted = sorted(range(len(midx_sums)), 
                             key=lambda k: midx_sums[k])
        # Sestavení výsledného pole
        midx_set = np.array([midx[midx_sorted[i]] for i in range(len(midx))])   
    return midx_set.astype(int)  # Převod na celá čísla

In [46]:
import numpy as np


def admissible_neighbors(index, index_set):
    """Vrátí přípustné sousední indexy pro daný index v monotonní množině indexů"""
    for_neighbors = forward_neighbors(index)  # Získá všechny "dopředné" sousedy
    # Filtruje jen přípustné sousedy (kde všechny zpětné sousedy jsou v index_set)
    for_truefalse = [is_admissible(fn, index_set) for fn in for_neighbors]
    adm_neighbors = np.array(for_neighbors)[for_truefalse].tolist()  # Konverze na seznam
    return adm_neighbors


def is_admissible(index, index_set):
    """Kontroluje, zda je index přípustný (všechny jeho zpětné sousedy obsahuje index_set)"""
    back_neighbors = backward_neighbors(index)  # Získá všechny "zpětné" sousedy
    for ind_b in back_neighbors:  # Pro každého zpětného souseda
        if ind_b not in index_set:  # Pokud není v množině, index není přípustný
            return False
    return True  # Všechny zpětné sousedy jsou v množině


def forward_neighbors(index):
    """Vrátí dopředné sousedy - každou dimenzi zvýší o 1 (např. (2,1) → (3,1), (2,2))"""
    N = len(index)  # Počet dimenzí multiindexu
    for_neighbors = []
    for i in range(N):  # Pro každou dimenzi
        index_tmp = index[:]  # Kopie původního indexu
        index_tmp[i] = index_tmp[i] + 1  # Inkrementace aktuální dimenze
        for_neighbors.append(index_tmp)  # Přidání do seznamu sousedů
    return for_neighbors


def backward_neighbors(index):
    """Vrátí zpětné sousedy - každou dimenzi sníží o 1 (pokud možno, např. (2,2) → (1,2), (2,1))"""
    N = len(index)  # Počet dimenzí multiindexu
    back_neighbors = []
    for i in range(N):  # Pro každou dimenzi
        index_tmp = index[:]  # Kopie původního indexu
        if index_tmp[i] > 0:  # Pokud lze snížit (nenulová hodnota)
            index_tmp[i] = index_tmp[i] - 1  # Dekrementace aktuální dimenze
            back_neighbors.append(index_tmp)  # Přidání do seznamu sousedů
    return back_neighbors

In [48]:
!pip install UQpy
!pip install --upgrade UQpy
!pip install torch

## **PCE** = OPENTURNS

In [50]:
# -*- coding: utf-8 -*-
import openturns as ot
import numpy as np
import scipy as sp

class PolynomialChaosExpansion():
    
    def __init__(self, pdf, exp_design_in, exp_design_out):
        """
        Inicializace PCE modelu
        
        Parametry:
        pdf - vstupní rozdělení pravděpodobnosti
        exp_design_in - vstupní data experimentu (2D numpy array)
        exp_design_out - výstupní data experimentu (2D numpy array)
        """
        
        # Pokud není zadáno rozdělení, identifikujeme ho z dat
        if pdf == None:
            chaos_algo_data = ot.FunctionalChaosAlgorithm(exp_design_in, 
                                                          exp_design_out)
            chaos_algo_data.run()
            self.pdf = chaos_algo_data.getDistribution()
        else:    
            self.pdf = pdf
            
        # Základní informace o modelu    
        self.num_inputs = self.pdf.getDimension()  # Počet vstupních proměnných
        self.num_outputs = exp_design_out.shape[1]  # Počet výstupních proměnných
        self.num_samples = exp_design_out.shape[0]  # Počet vzorků
        self.exp_design_inputs = exp_design_in  # Uložení vstupních dat
        self.exp_design_outputs = exp_design_out  # Uložení výstupních dat
        
        # Funkce pro generování multi-indexů
        self.enumerate_function = ot.LinearEnumerateFunction(self.num_inputs)
        
        # Kolekce ortogonálních polynomů pro každou vstupní proměnnou
        self.polynomial_collection = [
                  ot.StandardDistributionPolynomialFactory(
                      self.pdf.getMarginal(i))
                  for i in range(self.num_inputs)]
        
        # Vytvoření produktové báze polynomů
        self.product_basis = ot.OrthogonalProductPolynomialFactory(
                        self.polynomial_collection, self.enumerate_function)
        
        # Transformační funkce pro převod mezi rozděleními
        self.transformation = ot.DistributionTransformation(
                                self.pdf, self.product_basis.getMeasure())
    
    def set_multi_index_set(self, multi_index_set):
        """Nastavení multi-index množiny a převod na single-indexy"""
        self.multi_index_set = multi_index_set
        self.single_index_set = [self.enumerate_function.inverse(idx) 
                                  for idx in self.multi_index_set]
        
    def set_single_index_set(self, single_index_set):
        """Nastavení single-index množiny a převod na multi-indexy"""
        self.single_index_set = single_index_set
        self.multi_index_set = [list(self.enumerate_function(idx)) 
                                     for idx in self.single_index_set]
        
    def construct_basis(self):
        """Vytvoření polynomiální báze na základě nastavených indexů"""
        self.basis = self.product_basis.getSubBasis(self.single_index_set)
        self.num_polynomials = self.basis.getSize()  # Počet polynomů v bázi
        
    def set_exp_design(self, exp_design_in, exp_design_out):
        """Aktualizace experimentálních dat"""
        if self.num_outputs != exp_design_out.shape[1]:
            raise ValueError('Nesouhlasí dimenze výstupů!')
        self.exp_design_inputs = exp_design_in
        self.exp_design_outputs = exp_design_out 
        self.num_samples = exp_design_out.shape[0]
        
    def evaluate_basis(self, design_in):
        """Vyhodnocení bázových funkcí pro zadané vstupy"""
        n_samples, n_inputs = np.shape(design_in)
        if n_inputs != self.num_inputs:
            raise ValueError('Nesouhlasí dimenze vstupů!')
            
        # Transformace vstupních dat
        design_in_tf = np.array(self.transformation(design_in))
        
        # Vyhodnocení všech bázových funkcí
        eval_matrix = np.array([self.basis[j](design_in_tf) 
                                for j in range(self.num_polynomials)])
        
        # Odstranění nadbytečné dimenze
        eval_matrix = np.squeeze(eval_matrix)
        return eval_matrix.T  # Transpozice pro konzistentní tvar
    
    def compute_design_matrix(self):
        """Výpočet matice návrhu (vyhodnocení báze pro experimentální data)"""
        self.design_matrix = self.evaluate_basis(self.exp_design_inputs)
    
    def compute_coefficients(self):
        """Výpočet koeficientů PCE metodou nejmenších čtverců"""
        # Výpočet matice návrhu
        self.compute_design_matrix()
        
        # Řešení problému nejmenších čtverců
        self.coefficients, _, _, singular_values = np.linalg.lstsq(
                                                      self.design_matrix, 
                                                      self.exp_design_outputs,
                                                      rcond=None)
        # Výpočet čísla podmíněnosti
        self.condition_number = singular_values[0] / singular_values[-1]
    
    def predict(self, design_in):
        """Predikce výstupů pro nová vstupní data"""
        eval_matrix = self.evaluate_basis(design_in)
        return eval_matrix.dot(self.coefficients)  # Lineární kombinace
    
    def compute_mean(self):
        """Výpočet střední hodnoty (nulový koeficient)"""
        return self.coefficients[0]
    
    def compute_variance(self):
        """Výpočet rozptylu (součet čtverců ostatních koeficientů)"""
        return np.sum(np.square(self.coefficients[1:]), axis=0)
    
    def compute_sobol_first(self):
        """Výpočet Sobolových indexů prvního řádu"""
        sobol_f = np.empty([self.num_inputs, self.num_outputs])
        variance = self.compute_variance()
        
        # Odstranění nulového multi-indexu
        midx_minus_0 = np.delete(self.multi_index_set, 0, axis=0)
        
        for i in range(self.num_inputs):
            # Najdi řádky kde jsou všechny ostatní indexy nulové
            midx_minus_i = np.delete(midx_minus_0, i, axis=1)
            row_sum = np.sum(midx_minus_i, axis=1)
            zero_rows = np.asarray(np.where(row_sum==0)).flatten() + 1
            
            # Částečný rozptyl pro i-tou proměnnou
            partial_variance = np.sum(np.square(self.coefficients[zero_rows]),
                                      axis=0)
            sobol_f[i,:] = partial_variance / variance
        return sobol_f
    
    def compute_sobol_total(self):
        """Výpočet celkových Sobolových indexů"""
        sobol_t = np.empty([self.num_inputs, self.num_outputs])
        variance = self.compute_variance()
        mis = np.array(self.multi_index_set)
        
        for i in range(self.num_inputs):
            # Najdi řádky kde i-tý index není nula
            idx_column_i = mis[:,i] 
            non_zero_rows = np.asarray(np.where(idx_column_i!=0)).flatten()
            
            # Částečný rozptyl pro i-tou proměnnou
            partial_variance = np.sum(
                                np.square(self.coefficients[non_zero_rows]),
                                axis=0)
            sobol_t[i,:] = partial_variance / variance
        return sobol_t
    
    def compute_generalized_sobol_first(self):
        """Výpočet zobecněných Sobolových indexů 1. řádu"""
        variance = self.compute_variance()
        sobol_f = self.compute_sobol_first()
        
        # Částečné rozptyly pro každou proměnnou
        partial_variances = sobol_f*variance
        
        # Agregovaný rozptyl (pro všechny výstupy dohromady)
        aggregated_variance = np.sum(variance)
        
        # Součet částečných rozptylů přes všechny výstupy
        aggregated_partial_variances = np.sum(partial_variances, axis=1)
        
        # Zobecněné indexy
        sobol_f_gen = aggregated_partial_variances / aggregated_variance
        return sobol_f_gen

    def compute_generalized_sobol_total(self):
        """Výpočet zobecněných celkových Sobolových indexů"""
        variance = self.compute_variance()
        sobol_t = self.compute_sobol_total()
        partial_variances = sobol_t*variance
        aggregated_variance = np.sum(variance)
        aggregated_partial_variances = np.sum(partial_variances, axis=1)
        sobol_t_gen = aggregated_partial_variances / aggregated_variance
        return sobol_t_gen

In [51]:
# Testuje samotné PCE
"""
num_samples = 500
marginals = [Uniform(0, 5), Uniform(0, 2)] 
input_distribution = JointIndependent(marginals)
input_sample = input_distribution.rvs(num_samples)
print("Generovaná vstupní data:")
print(input_sample)

def test_function(X):
    x1, x2 = X[:, 0], X[:, 1]
    return np.expand_dims(2*x1 + 3*x2**2, axis=1)

output_sample = test_function(input_sample)

pce_uqpy = PolynomialChaosExpansionUQPY(input_distribution, input_sample, output_sample)

test_inputs = input_distribution.rvs(10)
predictions = pce_uqpy.predict(test_inputs)

mean_value = pce_uqpy.compute_mean()
variance_value = pce_uqpy.compute_variance()

sobol_first = pce_uqpy.compute_sobol_first()
sobol_total = pce_uqpy.compute_sobol_total()

print("První řád Sobolových indexů:")
print(sobol_first)
print("Celkové Sobolovy indexy:")
print(sobol_total)
print("Střední hodnota modelu:")
print(mean_value)
print("Rozptyl modelu:")
print(variance_value)

true_outputs = test_function(test_inputs)
print("Skutečné hodnoty vs Predikované hodnoty:")
for i in range(len(test_inputs)):
    print(f"Vstupy: {test_inputs[i]}, Skutečná hodnota: {true_outputs[i][0]}, Predikovaná hodnota: {predictions[i][0]}")
    
"""

'\nnum_samples = 500\nmarginals = [Uniform(0, 5), Uniform(0, 2)] \ninput_distribution = JointIndependent(marginals)\ninput_sample = input_distribution.rvs(num_samples)\nprint("Generovaná vstupní data:")\nprint(input_sample)\n\ndef test_function(X):\n    x1, x2 = X[:, 0], X[:, 1]\n    return np.expand_dims(2*x1 + 3*x2**2, axis=1)\n\noutput_sample = test_function(input_sample)\n\npce_uqpy = PolynomialChaosExpansionUQPY(input_distribution, input_sample, output_sample)\n\ntest_inputs = input_distribution.rvs(10)\npredictions = pce_uqpy.predict(test_inputs)\n\nmean_value = pce_uqpy.compute_mean()\nvariance_value = pce_uqpy.compute_variance()\n\nsobol_first = pce_uqpy.compute_sobol_first()\nsobol_total = pce_uqpy.compute_sobol_total()\n\nprint("První řád Sobolových indexů:")\nprint(sobol_first)\nprint("Celkové Sobolovy indexy:")\nprint(sobol_total)\nprint("Střední hodnota modelu:")\nprint(mean_value)\nprint("Rozptyl modelu:")\nprint(variance_value)\n\ntrue_outputs = test_function(test_inputs

## **SAPCE**

In [53]:
class SensitivityAdaptivePCE:
    def __init__(self, pdf, exp_design_in, exp_design_out, max_partial_degree=10, tolerance=1e-3):
        self.pdf = pdf if pdf is not None else JointIndependent([Uniform(0, 1) for _ in range(exp_design_in.shape[1])])
        self.exp_design_in = exp_design_in
        self.exp_design_out = exp_design_out
        self.max_partial_degree = max_partial_degree
        self.tolerance = tolerance   # Ukládání do self zajišťuje, že hodnoty přežijí mimo __init__ a jsou dostupné v celé třídě.

    
        num_inputs = self.pdf.getDimension() #Zjistí počet vstupních proměnných z pravděpodobnostního rozdělení 
        td1_set = td_multiindex_set(num_inputs, 1).tolist() #Vytvoří množinu multiindexů pro polynomy 1. stupně
        self.pce = PolynomialChaosExpansion(self.pdf, self.exp_design_in, self.exp_design_out) #Inicializuje PCE model, který bude aproximovat vztah mezi vstupy a výstupy
        self.pce.set_multi_index_set(td1_set) #Nastaví množinu multiindexů definovanou v  td1_set
        self.pce.construct_basis() #Sestaví ortogonální polynomiální bázi na základě
        self.pce.compute_coefficients() #Spočítá koeficienty PCE metodou nejmenších čtverců (LSQ) z dat.
         
        self.active_multi_indices = [self.pce.multi_index_set[0]]  #Začíná s nulovým multi-indexem
        self.admissible_multi_indices = self.pce.multi_index_set[1:] #Inicializace přípustných multi-indexůObsahuje všechny multi-indexy 1. stupně
        admissible_coefficients = self.pce.coefficients[1:].tolist() # Pro každý přípustný multi-index spočítá součet absolutních hodnot jeho koeficientů
        aggregated_admissible_coefficients = np.sum(np.abs(admissible_coefficients), axis=1) #absolutni hodnota, bez ohledu na zanemenk
        
        help_index = np.argmax(aggregated_admissible_coefficients) #Najde index (pozici) polynomu s největším součtem absolutních hodnot koeficientů
        max_admissible_multi_index = self.admissible_multi_indices.pop(help_index) #Odstrani a ulozi max. Algoritmus pak nemusí procházet již vybrané členy
        self.active_multi_indices.append(max_admissible_multi_index)  #Aktualizace aktivních multi-indexů
        
    def construct_adaptive_basis(self, max_condition_number=1e2, termination_info=True):
        while True:
            # T1: Kontroluje, zda číslo podmíněnosti aktuální matice překročilo limit
            if self.pce.condition_number > max_condition_number:
                if termination_info:
                    print("Adaptive basis construction terminated:" 
                        + " design matrix not sufficiently well-conditioned.")
                break
            
            # Hledání nových přípustných multi-indexů
            new_admissible_multi_indices = admissible_neighbors(
                                            self.active_multi_indices[-1],
                                            self.active_multi_indices)
            #print("new_admissible_multi_indices")
            #print("\033[95m" + str(new_admissible_multi_indices) + "\033[0m")
            # T2: Zajišťuje, že všechny nové přípustné multi-indexy splňují podmínku maximálního parciálního stupně, Stupně polynomu
    
            for idx, adm_multi_Indcs in reversed(list(enumerate(new_admissible_multi_indices))):
                for adm_multi_Indx in adm_multi_Indcs:
                    if adm_multi_Indx > self.max_partial_degree:
                        new_admissible_multi_indices.pop(idx)
                            
            #Kontroluje, zda číslo podmíněnosti aktuální matice překročilo limit
            if [self.max_partial_degree]*self.pce.num_inputs in self.active_multi_indices: 
                if len(new_admissible_multi_indices) == 0:
                    if termination_info:
                        print("Adaptive basis construction terminated:" 
                        + " maximum partial degree reached.")
                    break
            
            # Kontroluje, zda počet členů báze nepřekročil počet trénovacích vzorků
            num_terms = len(self.active_multi_indices) + \
                        len(self.admissible_multi_indices) +\
                        len(new_admissible_multi_indices)
            #print("num_terms")
            #print("\033[95m" + str(num_terms) + "\033[0m")
            
            if num_terms >= len(self.pce.exp_design_inputs):
                if termination_info:
                    print("Adaptive basis construction terminated:" 
                        + " basis cardinality reached experimental design size.")
                break
             
            
            self.admissible_multi_indices += new_admissible_multi_indices  #Přidá nově nalezené přípustné multi-indexy do stávající množiny přípustných indexů
            #print(" self.admissible_multi_indices")
            #print("\033[95m" + str( self.admissible_multi_indices) + "\033[0m")
            all_multi_indices = self.active_multi_indices + self.admissible_multi_indices
            #print("all_multi_indices")
            #print("\033[95m" + str(all_multi_indices) + "\033[0m")                                                       
            self.pce.set_multi_index_set(all_multi_indices) #Nastaví kompletní množinu indexů do PCE modelu
            #print("  self.pce.set_multi_index_set")
           # print("\033[95m" + str(  self.pce.set_multi_index_set) + "\033[0m")
            self.pce.construct_basis()
            #print("    self.pce.construct_basis()")
           # print("\033[95m" + str(self.pce.construct_basis)+ "\033[0m")
            self.pce.compute_coefficients()
            #print(" self.pce.compute_coefficients")
            #print("\033[95m" + str( self.pce.compute_coefficients) + "\033[0m")
            # Výběr a přesun nejvýznamnějšího indexu
            idx = len(self.active_multi_indices)
            #print("idx")
            #print("\033[95m" + str(idx) + "\033[0m")
            admissible_coefficients = self.pce.coefficients[idx:].tolist()
            #print("admissible_coefficients")
           # print("\033[95m" + str(admissible_coefficients) + "\033[0m")
            aggregated_admissible_coefficients = np.sum(np.abs(admissible_coefficients), axis=1)
            #print("aggregated_admissible_coefficients")
           # print("\033[95m" + str(aggregated_admissible_coefficients) + "\033[0m")
            help_index = np.argmax(aggregated_admissible_coefficients)
            #print("help_index")
            #print("\033[95m" + str(help_index) + "\033[0m")
            max_admissible_multi_index = self.admissible_multi_indices.pop(help_index)
           # print("max_admissible_multi_index")
            #print("\033[95m" + str(max_admissible_multi_index) + "\033[0m")
            self.active_multi_indices.append(max_admissible_multi_index)
            #print(" self.active_multi_indices.append")
            #print("\033[95m" + str(self.active_multi_indices.append) + "\033[0m")      

    def construct_active_pce(self):      ## Zacit zde
        pce = PolynomialChaosExpansion(self.pdf, 
                            self.exp_design_in, 
                            self.exp_design_out)
        pce.set_multi_index_set(self.active_multi_indices)
        pce.construct_basis()
        pce.compute_coefficients()
        return pce
    
    # nový polynomiální chaosový expanzní (PCE) model, který kombinuje aktuální aktivní i přípustné multi-indexy
    # Není to jen "srovnání" – přímo ovlivňuje sestavu týmu!
    def construct_augmented_pce(self):
        pce = PolynomialChaosExpansion(self.pdf, 
                                    self.exp_design_in, 
                                    self.exp_design_out)
        pce.set_multi_index_set(self.active_multi_indices + 
                            self.admissible_multi_indices)
        pce.construct_basis()
        pce.compute_coefficients()
        return pce      

    # Vytvoří optimalizovanou verzi rozšířeného PCE modelu s kontrolou stability.
    def construct_reduced_augmented_pce(self, max_condition_number=1e2):
        # compute augmented pce
        pce = self.construct_augmented_pce()
        while True:
            # exit if the condition number is acceptable
            if pce.condition_number <= max_condition_number and\
                len(pce.multi_index_set) <= len(pce.exp_design_inputs):
                break
            # remove single- and multi-index with minimum contribution
            idx_min = np.argmin(np.sum(np.abs(pce.coefficients), axis=1))
            pce.multi_index_set.pop(idx_min)
            pce.single_index_set.pop(idx_min)
            # compute pce basis and coefficients with reduced multi-index set
            pce.construct_basis()
            pce.compute_coefficients()
        # re-order multi-indices and coefficients
        # coeffs = pce.coefficients
        # midx = np.array(pce.multi_index_set)
        # coeffs_aggr = np.sum(np.abs(coeffs), axis=1)
        # order_idx = np.flip(np.argsort(coeffs_aggr))
        # midx_ord = midx[order_idx, :].tolist()
        # pce.set_multi_index_set(midx_ord)
        # pce.construct_basis()
        # pce.compute_coefficients()
        return pce

In [54]:
import openturns as ot
import numpy as np
import time
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_true, y_pred, multioutput='uniform_average'):
    mse = mean_squared_error(y_true, y_pred, multioutput=multioutput)
    if isinstance(mse, np.ndarray):
        return np.sqrt(mse)
    return np.sqrt(mse)

def test_SensitivityAdaptivePCE():
    # Nastavení parametrů
    dim = 3  # Počet vstupních proměnných
    size = 1000  # Počet trénovacích vzorků
    test_size = 50  # Počet testovacích vzorků
    
    # Vytvoření rovnoměrného rozdělení
    marginals = [ot.Uniform(0, 1) for _ in range(dim)]
    print(marginals)
    distribution = ot.ComposedDistribution(marginals)
    print(distribution)
    
    # Generování trénovacích dat
    experiment = ot.MonteCarloExperiment(distribution, size)
    exp_design_in = experiment.generate()
    
    # Testovací model s více výstupy
    def model(x):
        x_np = np.array(x)
        return np.column_stack((
            x_np[:, 0] + 2 * x_np[:, 1] + 3 * x_np[:, 2],
            x_np[:, 0] * x_np[:, 1] + x_np[:, 2],
            np.sin(x_np[:, 0]) + np.cos(x_np[:, 1]),
            np.exp(x_np[:, 2])
        ))
    
    exp_design_out = model(exp_design_in)
    
    # Generování testovacích dat
    test_experiment = ot.MonteCarloExperiment(distribution, test_size)
    test_in = test_experiment.generate()
    test_out = model(test_in)
    
    # Vytvoření a trénink adaptivního PCE modelu
    t0 = time.time()
    sapce = SensitivityAdaptivePCE(
        pdf=distribution, 
        exp_design_in=exp_design_in, 
        exp_design_out=exp_design_out, 
        max_partial_degree=10,
        tolerance=1e-3
    )
    sapce.construct_adaptive_basis(max_condition_number=1e3)
    pce = sapce.pce  # Předpokládáme, že třída SensitivityAdaptivePCE má tento atribut
    print("pce")
    print("\033[95m" + str(pce) + "\033[0m")
    
    # Předpovědi na testovacích datech
    pce_predictions = pce.predict(test_in)
    print("pce_predictions")
    print("\033[95m" + str(pce_predictions) + "\033[0m")
    # Vyhodnocení přesnosti
    vector_rmse = root_mean_squared_error(test_out, pce_predictions, multioutput='raw_values')
    avg_rmse = root_mean_squared_error(test_out, pce_predictions, multioutput='uniform_average')
    
    # Výpočet R² skóre
    r2 = []
    for j in range(test_out.shape[1]):
        ss_res = np.sum((test_out[:, j] - pce_predictions[:, j])**2)
        ss_tot = np.sum((test_out[:, j] - np.mean(test_out[:, j]))**2)
        r2.append(1 - ss_res/ss_tot)
    
    # Výstupy modelu
    pce_mean = pce.compute_mean()
    print("pce_mean")
    print("\033[95m" + str(pce_mean) + "\033[0m")

    pce_variance = pce.compute_variance()
    print("pce_variance")
    print("\033[95m" + str(pce_variance) + "\033[0m")
   
    pce_std = np.sqrt(pce_variance)
    print("pce_std")
    print("\033[95m" + str(pce_std) + "\033[0m")
# Spuštění testu
test_SensitivityAdaptivePCE()

[class=Uniform name=Uniform dimension=1 a=0 b=1, class=Uniform name=Uniform dimension=1 a=0 b=1, class=Uniform name=Uniform dimension=1 a=0 b=1]
ComposedDistribution(Uniform(a = 0, b = 1), Uniform(a = 0, b = 1), Uniform(a = 0, b = 1), IndependentCopula(dimension = 3))
Adaptive basis construction terminated: design matrix not sufficiently well-conditioned.
pce
pce_predictions
[[1.81444915 0.38567358 1.48983097 1.2391879 ]
 [4.31955127 1.25646754 1.36064272 1.90390011]
 [5.35807452 1.51590761 1.15321416 2.64493525]
 [4.84616748 1.55967374 1.43225413 1.99603068]
 [3.5659802  0.93635216 1.70903694 2.35737453]
 [2.77379745 0.58637966 1.08301318 1.50386089]
 [3.18307911 0.69519779 0.95466401 1.85312866]
 [4.56507777 0.96762463 0.69594928 2.50318392]
 [4.72173005 1.39105455 1.26000597 2.02374215]
 [2.64371503 0.38441394 0.79279381 1.14039049]
 [3.04673862 0.84250514 1.46427879 1.31082413]
 [3.68081062 0.89456211 1.12423656 1.83223567]
 [4.2933356  1.27068839 1.70682876 2.12081799]
 [3.8225492